In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import torch
import trimesh

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datareader import Ho3dReader

In [ ]:
video_id = 'AP14'

In [ ]:
sam_dir = f'/home/simonep01/sam-3d-objects/meshes/{video_id}'
output = torch.load(f'{sam_dir}/output_data.pt', map_location='cuda:0')

In [ ]:
from pytorch3d.transforms import quaternion_to_matrix, Transform3d

In [ ]:
_R_ZUP_TO_YUP = np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]], dtype=np.float32)
_R_YUP_TO_ZUP = _R_ZUP_TO_YUP.T
my_rotation = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]], dtype=np.float32)

In [ ]:
def compose_transform(
    scale: torch.Tensor, rotation: torch.Tensor, translation: torch.Tensor
) -> Transform3d:
    """
    Args:
        scale: (..., 3) tensor of scale factors
        rotation: (..., 3, 3) tensor of rotation matrices
        translation: (..., 3) tensor of translation vectors
    """
    tfm = Transform3d(dtype=scale.dtype, device=scale.device)
    return tfm.scale(scale).rotate(rotation).translate(translation)

In [ ]:
mesh = output['glb'].copy()
mesh.export(f'{sam_dir}/initial_mesh.obj')

In [ ]:
# 1. Convert from Y-up (OBJ) to Z-up (transform space)
vertices = mesh.vertices.astype(np.float32) @ _R_YUP_TO_ZUP
vertices_tensor = torch.from_numpy(vertices).float().to(output["rotation"].device)

# 2. Apply transformation in Z-up space
R_l2c = quaternion_to_matrix(output['rotation'])
l2c_transform = compose_transform(
    scale=output['scale'],
    rotation=R_l2c,
    translation=output['translation'],
)
vertices_transformed = l2c_transform.transform_points(vertices_tensor.unsqueeze(0))
mesh.vertices = vertices_transformed.squeeze(0).cpu().numpy() @ my_rotation

In [ ]:
mesh.export(f'{sam_dir}/transformed_mesh.obj')